In [1]:
import numpy as np
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
import glob
import os

import torch
#import torchvision
#import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data as D
from sklearn.model_selection import train_test_split
from skimage.color import rgb2gray
from skimage import measure, filters
from sklearn.preprocessing import OneHotEncoder
print(torch.__version__)
import time

save_name = '/home/jay/data/BNL_032020/BNL_032020_900_1/pytorchtest/singlelayer8_7x7batch128_short_blur3_sigmoid'

1.4.0


In [2]:
class single_layer(nn.Module):
    def __init__(self):
        super(single_layer,self).__init__()
        self.conv0 = nn.Conv2d(1,2,7,dilation=1,padding=3)
        self.b0 = nn.BatchNorm2d(2)
        
#        self.conv1 = nn.ConvTranspose2d(8,2,7,stride=2,padding=3)
#        self.b1 = nn.BatchNorm2d(2)
#
#        self.conv2 = nn.Conv2d(2,8,7,dilation=1,padding=3)
#        self.b2 = nn.BatchNorm2d(8)
        
        self.conv3 = nn.Conv2d(2,1,7,dilation=1,padding=3)
        self.b3 = nn.BatchNorm2d(1)
        
        self.pool = nn.MaxPool2d(2,2)

        
    def forward(self,x):
#        c0 = self.pool(self.b0(F.relu(self.conv0(x))))
#        c1 = self.b1(F.relu(self.conv1(c0,output_size=x.size())))
#        c2 = self.b2(F.relu(self.conv2(c1)))
#        c3 = self.b3(F.relu(self.conv3(c2)))
#        c1 = F.softmax(c1,dim=0)
        c0 = self.b0(F.relu(self.conv0(x)))
        c1 = self.b3(F.relu(self.conv3(c0)))
        #return c1
        return torch.sigmoid(c1)

model = single_layer()

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(),lr = 0.0005) 

#scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=[30])


device = torch.device('cuda:0,1,2,3' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
model = nn.DataParallel(model)
#model.to(device)
### Use the following line only if training on multiple GPUs
#model = nn.DataParallel(model)

### Use following section only if continuing training from a checkpoint
checkpoint = torch.load('/home/jay/data/BNL_032020/BNL_032020_900_1/pytorchtest/singlelayer8_7x7batch128_short_blur3_sigmoid_lr0001_e50.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

for state in optimizer.state.values():
    for k,v in state.items():
        if isinstance(v,torch.Tensor):
            state[k] = v.to(device)

### Change following line to model.eval() for inference
model.eval()
#model.to(device)
#model = nn.DataParallel(model)


cuda:0


DataParallel(
  (module): single_layer(
    (conv0): Conv2d(1, 2, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (b0): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(2, 1, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (b3): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [ ]:
#images = np.load('900C_driftcorrect.npy',mmap_mode = 'r+')
#def norm(im):
#    return (im - np.min(im)) / (np.max(im) - np.min(im))
#
#def scale(im):
#    return (im - np.mean(im)) / np.std(im)
#
#for i in range(len(images)):
#    if i % 100 == 0:
#        print(i)
#    images[i] = norm(scale(filters.gaussian(images[i],sigma=3)))
#np.save('900C_driftcorrect_gauss3.npy',images)

In [3]:
start = time.time()
print('Loading Images')
images = np.load('900C_driftcorrect_gauss3.npy',mmap_mode='r+').reshape(-1,1,3250,3250)
print('\t',images.shape)

#def norm(im):
#    return (im - np.min(im)) / (np.max(im) - np.min(im))
#
#print('Preprocessing')
#for i in range(len(images)):
#    if i % 100 == 0:
#        print('\timage {} / {}'.format(i,len(images)))
#    images[i] = norm(filters.gaussian(images[i],sigma=3))

batch_size=16
tensor_im = torch.stack([torch.Tensor(i) for i in images])
dataset = torch.utils.data.TensorDataset(tensor_im)
dataloader = torch.utils.data.DataLoader(dataset,batch_size=batch_size,shuffle=False)

print('Inference')
print('\tInitializing')
outputs = np.zeros_like(images)
print('\tEvaluating')
for i, data in enumerate(dataloader,0):
    if i % 5 == 0:
        print('\t {} / {}'.format(i*batch_size,len(images)))
    inputs = data[0]
    inputs = inputs.to(device)
    
    out = model(inputs).detach().cpu().numpy()
    outputs[i*batch_size:i*batch_size + batch_size] = out

print('Finished!')
print('\tTime: ',time.time() - start)

Loading Images
	 (800, 1, 3250, 3250)
Inference
	Initializing
	Evaluating
	 0 / 800
	 80 / 800
	 160 / 800
	 240 / 800
	 320 / 800
	 400 / 800
	 480 / 800
	 560 / 800
	 640 / 800
	 720 / 800
Finished!
	Time:  140.3405556678772


In [5]:
def norm(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

for i in range(len(outputs)):
    if i % 100 == 0:
        print(i)
    outputs[i] = norm(outputs[i])
    if np.min(outputs[i]) != 0:
        print('min: ',i)
    if np.max(outputs[i]) != 1:
        print('max: ',i)


np.save('900C_out.npy',outputs.reshape(-1,3250,3250))

0
100
200
300
400
500
600
700


In [ ]:
mins = []
maxes = []

def norm(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

for i in range(len(outputs)):
    outputs[i,0] = norm(outputs[i,0])
#    mins.append(np.min(outputs[i,0]))
#    maxes.append(np.max(outputs[i,0]))

In [ ]:
f,a = plt.subplots(figsize=(20,10),ncols=2)
a[0].hist(mins,color='blue')
a[1].hist(maxes,color='red')

In [ ]:
f,a = plt.subplots(figsize=(30,10),ncols=3)
n = int(np.random.rand()*len(outputs))
a[0].imshow(images[n,0],plt.cm.gray)
a[0].set_title('in')
a[1].imshow(outputs[n,0],plt.cm.gray)
a[1].set_title('out')
a[2].hist(images[n,0].reshape(-1,1))
a[2].hist(outputs[n,0].reshape(-1,1))